#### Importando as bibliotecas

In [1]:
import pandas as pd

#### Importando os dados

In [7]:
df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')

C:\Users\jose-\AppData\Local\Temp\ipykernel_13516\1792964528.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/TA_PRECO_MEDICAMENTO.CSV', sep = ';', encoding='latin-1', on_bad_lines='skip')


In [14]:
# Colunas do DataFrame

df.columns

Index(['substância', 'cnpj', 'laboratório', 'código ggrem', 'registro',
       'ean 1', 'ean 2', 'ean 3', 'produto', 'apresentação',
       'classe terapêutica', 'tipo de produto (status do produto)',
       'regime de preço', 'pf sem impostos', 'pf 0%', 'pf 12%', 'pf 17%',
       'pf 17% alc', 'pf 17,5%', 'pf 17,5% alc', 'pf 18%', 'pf 18% alc',
       'pf 20%', 'pmc 0%', 'pmc 12%', 'pmc 17%', 'pmc 17% alc', 'pmc 17,5%',
       'pmc 17,5% alc', 'pmc 18%', 'pmc 18% alc', 'pmc 20%',
       'restrição hospitalar', 'cap', 'confaz 87', 'icms 0%',
       'análise recursal',
       'lista de concessão de crédito tributário (pis/cofins)',
       'comercialização 2022', 'tarja'],
      dtype='object')

In [13]:
# Deixa os nomes das colunas em letras minúsculas
df.rename(columns = {column:column.lower() for column in df.columns}, inplace = True)

### Capturando as colunas importantes
<b>Substância</b>: princípio ativo do medicamento.<br>
<b>Produto</b>: nome comercial.<br>
<b>Apresentação</b>: descrição de como o medicamento é apresentado, quanto a sua 
forma farmacêutica, dosagem e quantidade.<br>
<b>Classe terapêutica</b>: está relacionado com a finalidade e o local de atuação do medicamento.<br>
<b>Tipo de produto (status do produto)</b>: Indica qual se a venda do medicamento necessita de recita.<br>
<b>Tarja</b>: indica qual se a venda do medicamento necessita de recita, sendo classificado em Tarja preta, Tarja vermelha ou Venda livre.<br>
<b>PF 18% alc</b>: preço de fábrica.<br>
<b>PMC 18% alc</b>: preço máximo ao consumidor (variável alvo).<br>
<br>
<b>Obs:</b> Foram escolhidos os PF e PMC com alíquota de 18% ao invés dos outros, pois este é o percentual aplicado na maioria dos estados brasileiros.

In [22]:
df = df[['substância', 'produto', 'apresentação',
       'classe terapêutica', 'tipo de produto (status do produto)',
       'pf 18% alc','pmc 18% alc', 'tarja']]